In [1]:
import pandas as pd
import numpy as np

import neologdn
import MeCab

import re

from gensim.models.doc2vec import Doc2Vec
from gensim.models.doc2vec import TaggedDocument

C:\Users\Tomori kengo\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


### サンプルデータの読み込み

In [2]:
#encoding='utf-8'で上手くいった
diary_df = pd.read_csv('SBI_Financial statement_201903.csv', encoding='utf-8')

In [3]:
diary_df

,text
0,国内経済が緩やかに回復するなか、マーケット環境は、米国と中国の貿易摩擦問題に対する警戒感等か...
1,このような環境の中、当社業績においては、ホールセールビジネスの拡大、トレーディング収益や金融...
2,当社は引続き他社を大きく上回る高いシェアを維持し、35.9％のシェアを獲得。
3,先物・オプションの委託個人売買代金シェアは、引き続き高水準を維持。
4,投資信託残高の四半期末残高は過去最高を更新し、信託報酬は高水準を維持。
5,2018年4月から2018年6月までの上場会社数は20社。
6,同期間のSBI証券引受関与率は100％と 引き続き業界トップ。


### MeCabにかける前準備としてneologdn.normalize()を使用して文章全体を正規化

In [4]:
def get_diary_normalization(text):
    diary_normalization = neologdn.normalize(text)
    return diary_normalization

### MeCab + neologdで形態素解析し、品詞で単語を絞った後に出現頻度0以上の単語を抽出してリストに格納する

In [5]:
def get_mecabed_word_list(diary_normalization):
    text_normalization = neologdn.normalize(diary_normalization)
    neologd_tagger = MeCab.Tagger('-Ochasen -d C:\mecab-ipadic-neologd')
    
    # neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して表示される
    # 原形、品詞などの間には「\t」が、分かち書きされた単語と単語の区切りには「\n」が表示される
    # 例: '空い\tアイ\t空く\t動詞-自立\t五段・カ行イ音便\t連用タ接続\nた\tタ\tた\t助動詞\t特殊・タ\t基本形\n時間....
    # まずはparse()で分かち書きした単語群は1つの文字列型になっているので「\n」で区切り、リスト型にする
    wakati_text_list = neologd_tagger.parse(text_normalization).split('\n')
    # 「\n」で区切り、リスト型にした結果の例は下記
    # ['空い\tアイ\t空く\t動詞-自立\t五段・カ行イ音便\t連用タ接続', ：リスト0番目
    # 'た\tタ\tた\t助動詞\t特殊・タ\t基本形',：リスト1番目
    # '時間\tジカン\t時間\t名詞-副詞可能\t\t',：リスト2番目
    
    ##【形態素解析結果を格納したリストから特定の品詞（品詞詳細部分まで考慮に入れた場合）のみ抽出】
    # 抽出したい品詞のリストを作成（完全一致）
    # 品詞参考URL：http://miner.hatenablog.com/entry/323
    hinshi_list = ['名詞-一般', '名詞-形容動詞語幹', '名詞-固有名詞-一般',  '名詞-サ変接続', '形容詞-自立', '形容詞-接尾', '形容詞-非自立', '動詞-自立', '動詞-接尾', '動詞-非自立', '副詞-一般', '副詞-助詞類接続']
    # hinshi_list = ('名詞-一般', '名詞-サ変接続', '名詞-固有名詞', '名詞-形容動詞語幹'...)とタプルでも同じ結果
    original_form_list =[]# 単語の「原形」のみ格納する(品詞情報は単語頻度を求める際に必要ないので除外)
    
    # parse() の出力結果の最後は「EOS」という文字のみ
    # EOSのとき、pos = wakati.split('\t')[3]の要素はないので下記forループを実行すると「list index out of range」とエラーを発生させてしまう
    # よってEOSのときは条件分岐if~breakでforループから抜け出すよう記述
    for wakati in wakati_text_list:
        surface = wakati.split('\t')[0]
        if surface == 'EOS':
            break
        else:
            pos = wakati.split('\t')[3]
            if pos in hinshi_list:# posはhinshi_listの中の要素と完全一致していないと抽出できない
                original_form_list.append(wakati.split('\t')[2])
    # ここまでで品詞によって単語を絞った
    
    ##【単語の出現頻度を求め、出現頻度0以上の単語のみ抽出】
    # まずは単語の出現頻度を求める
    import collections
    count = collections.Counter(original_form_list)
    word_count_list = count.most_common()
    word_count_list# word_count_listは全体がリスト型でリスト内に('する', 4), ('ぼんやり', 3),,,,といった単語とその出現頻度がタプルで格納されている
    # 次に出現頻度が0以上（今回はテキストデータが少ないので）の単語のみ抽出する
    Frequency1_word_list = []
    for word_count in word_count_list:
        if word_count[1] > 0:
            Frequency1_word_list.append(word_count[0])
    
    return Frequency1_word_list

### 素性ベクトル（文章の特徴を表現したベクトル）への変換

In [6]:
# diary_dfのtextカラムのすべての文章に「get_mecabed_word_list」関数（形態素解析する関数）を適応させたい

# apply関数の引数に関数を引き渡すことが可能、関数を全列に適応できる
#  参考URL：https://teratail.com/questions/134846
# diary_dfに関数を全列新たなカラム['Wakati']を追加する
# 参考URL：https://datumstudio.jp/blog/%e3%80%90%e7%89%b9%e5%88%a5%e9%80%a3%e8%bc%89%e3%80%91-%e3%81%95%e3%81%81%e3%80%81%e8%87%aa%e7%84%b6%e8%a8%80%e8%aa%9e%e5%87%a6%e7%90%86%e3%82%92%e5%a7%8b%e3%82%81%e3%82%88%e3%81%86%ef%bc%81-3
diary_df['Wakati'] = diary_df['text'].apply(get_mecabed_word_list)

In [7]:
diary_df

,text,Wakati
0,国内経済が緩やかに回復するなか、マーケット環境は、米国と中国の貿易摩擦問題に対する警戒感等か...,"[国内, 経済, 緩やか, 回復, する, マーケット, 環境, 貿易摩擦, 警戒感, 一進..."
1,このような環境の中、当社業績においては、ホールセールビジネスの拡大、トレーディング収益や金融...,"[収益, 環境, 当社, 業績, ホールセール, ビジネス, 拡大, トレーディング, 金融..."
2,当社は引続き他社を大きく上回る高いシェアを維持し、35.9％のシェアを獲得。,"[シェア, 当社, 引続き, 他社, 大きい, 上回る, 高い, 維持, する, 35.9%..."
3,先物・オプションの委託個人売買代金シェアは、引き続き高水準を維持。,"[先物, オプション, 委託, 個人, 売買代金, シェア, 引き続き, 高水準, 維持]"
4,投資信託残高の四半期末残高は過去最高を更新し、信託報酬は高水準を維持。,"[残高, 投資信託, 四半期, 過去最高, 更新, する, 信託報酬, 高水準, 維持]"
5,2018年4月から2018年6月までの上場会社数は20社。,"[2018年, 4月, 6月, 上場会社]"
6,同期間のSBI証券引受関与率は100％と 引き続き業界トップ。,"[期間, SBI証券, 引受, 関与, 引き続き, 業界, トップ]"


In [8]:
# Wakatiカラムの1行毎に格納されている単語はリストに格納されているのでリスト型と判断されCountVectorizer.fit_transform()が使用できない
# そこでjoin関数でリストの中身を文字列に変換する
# ただし、リストに格納されてる要素に数値が混ざってる場合はmap([適用関数], [対象リスト])を使う必要あり
# 参考URL：https://www.okadalabo.com/python%E3%83%AA%E3%82%B9%E3%83%88%E3%81%AE%E4%B8%AD%E8%BA%AB%E3%82%92%E6%96%87%E5%AD%97%E5%88%97%E3%81%AB%E5%A4%89%E6%8F%9B%E3%81%99%E3%82%8B%E3%80%82/
for i in range(0, len(diary_df)):
    diary_df['Wakati'][i] = ','.join((map(str, diary_df['Wakati'][i] )))

In [9]:
diary_df

,text,Wakati
0,国内経済が緩やかに回復するなか、マーケット環境は、米国と中国の貿易摩擦問題に対する警戒感等か...,"国内,経済,緩やか,回復,する,マーケット,環境,貿易摩擦,警戒感,一進一退,展開"
1,このような環境の中、当社業績においては、ホールセールビジネスの拡大、トレーディング収益や金融...,"収益,環境,当社,業績,ホールセール,ビジネス,拡大,トレーディング,金融,増加,前年同期,..."
2,当社は引続き他社を大きく上回る高いシェアを維持し、35.9％のシェアを獲得。,"シェア,当社,引続き,他社,大きい,上回る,高い,維持,する,35.9%,獲得"
3,先物・オプションの委託個人売買代金シェアは、引き続き高水準を維持。,"先物,オプション,委託,個人,売買代金,シェア,引き続き,高水準,維持"
4,投資信託残高の四半期末残高は過去最高を更新し、信託報酬は高水準を維持。,"残高,投資信託,四半期,過去最高,更新,する,信託報酬,高水準,維持"
5,2018年4月から2018年6月までの上場会社数は20社。,"2018年,4月,6月,上場会社"
6,同期間のSBI証券引受関与率は100％と 引き続き業界トップ。,"期間,SBI証券,引受,関与,引き続き,業界,トップ"


### BoW(Bag of Words)の実装

In [10]:
from sklearn.feature_extraction.text import CountVectorizer
CV = CountVectorizer()
# CountVectorizerを利用して単語と列番号の対応付けを実行 Document-Term Matrixを獲得できる
feature_vectors = CV.fit_transform(diary_df['Wakati'])
# どの単語を学習しているのかをCV.get_feature_names()で確認
vocabulary = CV.get_feature_names()

In [11]:
print(feature_vectors)

  (0, 30)	1
  (0, 13)	1
  (0, 47)	1
  (0, 48)	1
  (0, 43)	1
  (0, 12)	1
  (0, 5)	1
  (0, 23)	1
  (0, 46)	1
  (0, 44)	1
  (0, 24)	1
  (1, 50)	1
  (1, 26)	1
  (1, 20)	1
  (1, 25)	1
  (1, 51)	1
  (1, 9)	1
  (1, 36)	1
  (1, 10)	1
  (1, 11)	1
  (1, 40)	1
  (1, 34)	1
  (1, 21)	1
  (1, 43)	1
  (2, 42)	1
  :	:
  (3, 29)	1
  (3, 6)	1
  (3, 19)	1
  (3, 45)	1
  (3, 7)	1
  (4, 17)	1
  (4, 37)	1
  (4, 49)	1
  (4, 22)	1
  (4, 35)	1
  (4, 41)	1
  (4, 54)	1
  (4, 45)	1
  (4, 5)	1
  (5, 15)	1
  (5, 3)	1
  (5, 2)	1
  (5, 0)	1
  (6, 8)	1
  (6, 39)	1
  (6, 52)	1
  (6, 32)	1
  (6, 4)	1
  (6, 38)	1
  (6, 31)	1


In [12]:
print('今回の文章で学習した単語の数は{}語です。'.format(len(CV.get_feature_names())))
# print('今回の文章で学習した単語の数は{}語です。'.format(len(CV.vocabulary_.keys()))) 上記はこの表現でも同じ
print('--------------------------------------------------------------------------------------------')
print(vocabulary)# vocabulary = CV.get_feature_names()
print('--------------------------------------------------------------------------------------------')
print(CV.vocabulary_)
print('--------------------------------------------------------------------------------------------')
print(CV.vocabulary_.keys())
print('--------------------------------------------------------------------------------------------')
print(CV.vocabulary_.values())

今回の文章で学習した単語の数は55語です。
--------------------------------------------------------------------------------------------
['2018年', '35', '4月', '6月', 'sbi証券', 'する', 'オプション', 'シェア', 'トップ', 'トレーディング', 'ビジネス', 'ホールセール', 'マーケット', '一進一退', '上回る', '上場会社', '他社', '信託報酬', '個人', '先物', '前年同期', '収益', '四半期', '回復', '国内', '増加', '増収増益', '売買代金', '大きい', '委託', '展開', '引き続き', '引受', '引続き', '当社', '投資信託', '拡大', '更新', '期間', '業界', '業績', '残高', '獲得', '環境', '経済', '維持', '緩やか', '警戒感', '貿易摩擦', '過去最高', '達成', '金融', '関与', '高い', '高水準']
--------------------------------------------------------------------------------------------
{'国内': 24, '経済': 44, '緩やか': 46, '回復': 23, 'する': 5, 'マーケット': 12, '環境': 43, '貿易摩擦': 48, '警戒感': 47, '一進一退': 13, '展開': 30, '収益': 21, '当社': 34, '業績': 40, 'ホールセール': 11, 'ビジネス': 10, '拡大': 36, 'トレーディング': 9, '金融': 51, '増加': 25, '前年同期': 20, '増収増益': 26, '達成': 50, 'シェア': 7, '引続き': 33, '他社': 16, '大きい': 28, '上回る': 14, '高い': 53, '維持': 45, '35': 1, '獲得': 42, '先物': 19, 'オプション': 6, '委託': 29, '個人': 18, '売買代金': 27, '引き続き': 31

In [13]:
print(feature_vectors.toarray())

[[0 0 0 0 0 1 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 1 0 0 0 0 0
  0 0 0 0 0 0 0 1 1 0 1 1 1 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 1 1 0 0 0 1 1 0 0 0 0 0 0 0 1 0
  1 0 0 0 1 0 0 1 0 0 0 0 0 0 1 1 0 0 0]
 [0 1 0 0 0 1 0 1 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 1 0
  0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 1 0 1 0 1 0 0 0 0
  0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1]
 [0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1
  0 1 0 0 0 1 0 0 0 1 0 0 0 1 0 0 0 0 1]
 [1 0 1 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0
  0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0]]


In [14]:
# 疎行列をデータに、インデックスを元のテキストに、カラムを「文章から抽出した単語」したデータフレームを作成
# 書籍「Pythonによるあたらしいデータ分析の教科書 P.290」が参考になるかも
# columns=CV.vocabulary_.keys()にすると単語の並びとfeature_vectors.toarray()の0,1の並びとが対応できていないのでcolumnsにはCV.get_feature_names()を指定する
bow_df = pd.DataFrame(feature_vectors.toarray(), index=diary_df['text'],  columns=vocabulary)

In [15]:
bow_df

,2018年,35,4月,6月,sbi証券,する,オプション,シェア,トップ,トレーディング,...,維持,緩やか,警戒感,貿易摩擦,過去最高,達成,金融,関与,高い,高水準
text,,,,,,,,,,,,,,,,,,,,,
国内経済が緩やかに回復するなか、マーケット環境は、米国と中国の貿易摩擦問題に対する警戒感等から一進一 退の展開。,0,0,0,0,0,1,0,0,0,0,...,0,1,1,1,0,0,0,0,0,0
このような環境の中、当社業績においては、ホールセールビジネスの拡大、トレーディング収益や金融 収益の増加により前年同期比で増収増益を達成。,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,1,1,0,0,0
当社は引続き他社を大きく上回る高いシェアを維持し、35.9％のシェアを獲得。,0,1,0,0,0,1,0,1,0,0,...,1,0,0,0,0,0,0,0,1,0
先物・オプションの委託個人売買代金シェアは、引き続き高水準を維持。,0,0,0,0,0,0,1,1,0,0,...,1,0,0,0,0,0,0,0,0,1
投資信託残高の四半期末残高は過去最高を更新し、信託報酬は高水準を維持。,0,0,0,0,0,1,0,0,0,0,...,1,0,0,0,1,0,0,0,0,1
2018年4月から2018年6月までの上場会社数は20社。,1,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
同期間のSBI証券引受関与率は100％と 引き続き業界トップ。,0,0,0,0,1,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0


In [16]:
# to_csv関数でindexにカラム名を名付けるには「index_label='カラム名'」 を引数に追加する
bow_df.to_csv('bow_df.csv', index_label='text', encoding='shift-jis')

# TF-IDFの実装

In [17]:
# TF-IDFを作成するためのライブラリ
from sklearn.feature_extraction.text import TfidfVectorizer
# 初期化
TV = TfidfVectorizer()
# TfidfVectorizerを利用して単語と列番号の対応付けを実行 Document-Term Matrixを獲得できる
tfidf_feature_vectors = TV.fit_transform(diary_df['Wakati'])
# どの単語を学習しているのかをTV.get_feature_names()で確認
tfidf_vocabulary = TV.get_feature_names()

In [18]:
# dict(zip(TfidfVectorizer.get_feature_names(), TfidfVectorizer.idf_))
print(dict(zip(tfidf_vocabulary, TV.idf_)))
print('--------------------------------------------------------------------------------------------')
print(tfidf_feature_vectors)

{'2018年': 2.386294361119891, '35': 2.386294361119891, '4月': 2.386294361119891, '6月': 2.386294361119891, 'sbi証券': 2.386294361119891, 'する': 1.6931471805599454, 'オプション': 2.386294361119891, 'シェア': 1.9808292530117262, 'トップ': 2.386294361119891, 'トレーディング': 2.386294361119891, 'ビジネス': 2.386294361119891, 'ホールセール': 2.386294361119891, 'マーケット': 2.386294361119891, '一進一退': 2.386294361119891, '上回る': 2.386294361119891, '上場会社': 2.386294361119891, '他社': 2.386294361119891, '信託報酬': 2.386294361119891, '個人': 2.386294361119891, '先物': 2.386294361119891, '前年同期': 2.386294361119891, '収益': 2.386294361119891, '四半期': 2.386294361119891, '回復': 2.386294361119891, '国内': 2.386294361119891, '増加': 2.386294361119891, '増収増益': 2.386294361119891, '売買代金': 2.386294361119891, '大きい': 2.386294361119891, '委託': 2.386294361119891, '展開': 2.386294361119891, '引き続き': 1.9808292530117262, '引受': 2.386294361119891, '引続き': 2.386294361119891, '当社': 1.9808292530117262, '投資信託': 2.386294361119891, '拡大': 2.386294361119891, '更新': 2.386294361119891, 

In [19]:
print('今回の文章で学習した単語の数は{}語です。'.format(len(TV.get_feature_names())))
# print('今回の文章で学習した単語の数は{}語です。'.format(len(TV.vocabulary_.keys()))) 上記はこの表現でも同じ
print('--------------------------------------------------------------------------------------------')
print(tfidf_vocabulary)# vocabulary = CV.get_feature_names()
print('--------------------------------------------------------------------------------------------')
print(TV.vocabulary_)
print('--------------------------------------------------------------------------------------------')
print(TV.vocabulary_.keys())
print('--------------------------------------------------------------------------------------------')
print(TV.vocabulary_.values())

今回の文章で学習した単語の数は55語です。
--------------------------------------------------------------------------------------------
['2018年', '35', '4月', '6月', 'sbi証券', 'する', 'オプション', 'シェア', 'トップ', 'トレーディング', 'ビジネス', 'ホールセール', 'マーケット', '一進一退', '上回る', '上場会社', '他社', '信託報酬', '個人', '先物', '前年同期', '収益', '四半期', '回復', '国内', '増加', '増収増益', '売買代金', '大きい', '委託', '展開', '引き続き', '引受', '引続き', '当社', '投資信託', '拡大', '更新', '期間', '業界', '業績', '残高', '獲得', '環境', '経済', '維持', '緩やか', '警戒感', '貿易摩擦', '過去最高', '達成', '金融', '関与', '高い', '高水準']
--------------------------------------------------------------------------------------------
{'国内': 24, '経済': 44, '緩やか': 46, '回復': 23, 'する': 5, 'マーケット': 12, '環境': 43, '貿易摩擦': 48, '警戒感': 47, '一進一退': 13, '展開': 30, '収益': 21, '当社': 34, '業績': 40, 'ホールセール': 11, 'ビジネス': 10, '拡大': 36, 'トレーディング': 9, '金融': 51, '増加': 25, '前年同期': 20, '増収増益': 26, '達成': 50, 'シェア': 7, '引続き': 33, '他社': 16, '大きい': 28, '上回る': 14, '高い': 53, '維持': 45, '35': 1, '獲得': 42, '先物': 19, 'オプション': 6, '委託': 29, '個人': 18, '売買代金': 27, '引き続き': 31

In [20]:
print(tfidf_feature_vectors.toarray())

[[0.         0.         0.         0.         0.         0.22224442
  0.         0.         0.         0.         0.         0.
  0.3132277  0.3132277  0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.3132277
  0.3132277  0.         0.         0.         0.         0.
  0.3132277  0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.26000589 0.3132277  0.         0.3132277  0.3132277
  0.3132277  0.         0.         0.         0.         0.
  0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.28423219 0.28423219 0.28423219
  0.         0.         0.         0.         0.         0.
  0.         0.         0.28423219 0.28423219 0.         0.
  0.         0.28423219 0.28423219 0.         0.         0.
  0.         0.         0.         0.         0.23593713 0.
  0.28423219 0.         0.         0.         0.28423219

In [21]:
# 疎行列をデータに、インデックスを元のテキストに、カラムを「文章から抽出した単語」したデータフレームを作成
# 書籍「Pythonによるあたらしいデータ分析の教科書 P.290」が参考になるかも
# columns=TV.vocabulary_.keys()にすると単語の並びとtfidf_feature_vectors.toarray()の0,1の並びとが対応できていないのでcolumnsにはTV.get_feature_names()を指定する
tfidf_df = pd.DataFrame(tfidf_feature_vectors.toarray(), index=diary_df['text'],  columns=tfidf_vocabulary)

In [22]:
tfidf_df

,2018年,35,4月,6月,sbi証券,する,オプション,シェア,トップ,トレーディング,...,維持,緩やか,警戒感,貿易摩擦,過去最高,達成,金融,関与,高い,高水準
text,,,,,,,,,,,,,,,,,,,,,
国内経済が緩やかに回復するなか、マーケット環境は、米国と中国の貿易摩擦問題に対する警戒感等から一進一 退の展開。,0.0,0.000000,0.0,0.0,0.00000,0.222244,0.000000,0.000000,0.00000,0.000000,...,0.000000,0.313228,0.313228,0.313228,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000
このような環境の中、当社業績においては、ホールセールビジネスの拡大、トレーディング収益や金融 収益の増加により前年同期比で増収増益を達成。,0.0,0.000000,0.0,0.0,0.00000,0.000000,0.000000,0.000000,0.00000,0.284232,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.284232,0.284232,0.00000,0.000000,0.000000
当社は引続き他社を大きく上回る高いシェアを維持し、35.9％のシェアを獲得。,0.0,0.326425,0.0,0.0,0.00000,0.231609,0.000000,0.270961,0.00000,0.000000,...,0.231609,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.326425,0.000000
先物・オプションの委託個人売買代金シェアは、引き続き高水準を維持。,0.0,0.000000,0.0,0.0,0.00000,0.000000,0.363443,0.301689,0.00000,0.000000,...,0.257873,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.301689
投資信託残高の四半期末残高は過去最高を更新し、信託報酬は高水準を維持。,0.0,0.000000,0.0,0.0,0.00000,0.255765,0.000000,0.000000,0.00000,0.000000,...,0.255765,0.000000,0.000000,0.000000,0.360471,0.000000,0.000000,0.00000,0.000000,0.299222
2018年4月から2018年6月までの上場会社数は20社。,0.5,0.000000,0.5,0.5,0.00000,0.000000,0.000000,0.000000,0.00000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000
同期間のSBI証券引受関与率は100％と 引き続き業界トップ。,0.0,0.000000,0.0,0.0,0.38665,0.000000,0.000000,0.000000,0.38665,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.38665,0.000000,0.000000


In [23]:
# to_csv関数でindexにカラム名を名付けるには「index_label='カラム名'」 を引数に追加する
tfidf_df.to_csv('tfidf_df.csv', index_label='text', encoding='shift-jis')